In [1]:
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification
import torch

/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/dante/.pyenv/versions/3.12.0/lib/python3.12/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [7]:
veracity_path = "classes/models/veracity.pth"
sentiment_path = "classes/models/sentiment.pth"
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'

In [3]:
def dict_cleaner(model_state_dict):
        model_state_clean = {}
        for key, value in model_state_dict.items():
            if key == "fc.weight" or key == "fc.bias":
                key = key.replace("fc", "classifier")
            model_state_clean[key] = value
        return model_state_clean

In [4]:
def load_model(model_path,model_name):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        state_dict = torch.load(model_path,map_location= device)
        state_clean = dict_cleaner(state_dict)   
        model = BertForSequenceClassification.from_pretrained(model_name)
        model.load_state_dict(state_clean)   
        tokenizer = BertTokenizer.from_pretrained(model_name)
        return model,tokenizer
        

In [5]:
def save_model(model,tokenizer,save_path):
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

In [6]:
veracity_model,veracity_tokenizer= load_model(veracity_path,model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
sentiment_model,sentiment_tokenizer= load_model(sentiment_path,model_name)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
save_model(veracity_model,veracity_tokenizer,"veracity-model")

In [ ]:
save_model(sentiment_model,sentiment_tokenizer,"sentiment-model")

In [9]:
def upload_model(model,tokenizer,repo_name):
        model.push_to_hub(repo_name)
        tokenizer.push_to_hub(repo_name)

In [10]:
veracity_repo_name= "VerificadoProfesional/SaBERT-Spanish-Fake-News"
upload_model(veracity_model,veracity_tokenizer,veracity_repo_name)

model.safetensors: 100%|██████████| 439M/439M [30:59<00:00, 236kB/s]    


In [11]:
sentiment_repo_name= "VerificadoProfesional/SaBERT-Spanish-Sentiment-Analysis"
upload_model(sentiment_model,sentiment_tokenizer,sentiment_repo_name)

model.safetensors: 100%|██████████| 439M/439M [16:00<00:00, 458kB/s]
